In [ ]:
%load_ext autoreload
%autoreload 2

from nnTrain import train_efn, train_pfn, get_data, get_data_softDrop
import matplotlib.pyplot as plt
import numpy as np
from multiprocessing import Process
from tensorflow import keras
from keras.optimizers import Adam
from tensorflow import keras
from keras import backend as K
from energyflow.utils import data_split, to_categorical
from sklearn.model_selection import train_test_split
import gc
from tensorflow.keras.losses import BinaryCrossentropy

# define rng seeds for train/test/val split for replicable results
rng1 = 45446
rng2 = 25135

In [ ]:
import energyflow as ef
ef.__file__

# Hadron Level

In [ ]:
base="/uscms/home/sbrightt/nobackup/jets-ml/datasets/safeIncalculable_v2/"
nmax = 200000

hdata = get_data('q',hlevel=True,efrac=True,base=base,nmax=nmax,wta=True)
htrain, htest = train_test_split(hdata,train_size=0.5,random_state=rng1)
htest, hval = train_test_split(htest,train_size=0.5,random_state=rng2)

pdata = get_data('g',hlevel=True,efrac=True,base=base,nmax=nmax,wta=True)
ptrain, ptest = train_test_split(pdata,train_size=0.5,random_state=rng1)
ptest, pval = train_test_split(ptest,train_size=0.5,random_state=rng2)

train = np.concatenate((htrain,ptrain),axis=0)
train_labels = np.concatenate((np.ones((htrain.shape[0],1)),np.zeros((ptrain.shape[0],1))),axis=0)
perm = np.random.permutation(train.shape[0])
train = train[perm]
train_labels = train_labels[perm]

test = np.concatenate((htest,ptest),axis=0)
test_labels = np.concatenate((np.ones((htest.shape[0],1)),np.zeros((ptest.shape[0],1))),axis=0)
perm = np.random.permutation(test.shape[0])
test = test[perm]
test_labels = test_labels[perm]

val = np.concatenate((hval,pval),axis=0)
val_labels = np.concatenate((np.ones((hval.shape[0],1)),np.zeros((pval.shape[0],1))),axis=0)
perm = np.random.permutation(val.shape[0])
val = val[perm]
val_labels = val_labels[perm]

del hdata, pdata, htrain, htest, hval, ptrain, ptest, pval, perm

In [ ]:
# spectral EFN
L = 200.0
loss = BinaryCrossentropy(from_logits=True)
#out_act = keras.activations.linear
model_kwargs = {"Phi_sizes":(100,100,128), "F_sizes":(100,100,100),
                "F_acts":"relu","Phi_acts":"relu",
                "F_dropouts":0.1,
                "input_dim":2,
                "output_dim":1,
                "output_act":"linear",
                "patience":5,
                "optimizer":Adam(1e-3),
                "loss":loss,
                "summary":True}
model_kwargs['spectral'] = True
model_kwargs['lip_const'] = float(L)
model_kwargs['eps_bjorck'] = None
model_kwargs['bound_only'] = True
train_kwargs = {"epochs":200,"verbose":1,"batch_size":10000}
efn, auc, efn_fp, efn_tp, threshs = train_efn((train,train_labels),
                                              (test,test_labels),
                                              (val,val_labels),
                                              model_kwargs,
                                              train_kwargs,
                                              plot=True)
#efn.save(f"keras_models_wta_quarkGluon/spectralEFN_L{int(L)}_zpT_hadlevel.keras")
del efn
gc.collect()

In [ ]:
# spectral EFN L >= 1
#Ls = [20,30,40,50,60,70,80,90,
#      100,200,300,400,500,600,700,800,900,
#      1000,5000,10000]
#Ls = [2000,3000,4000,6000,7000,8000,9000]
Ls = [2,3,4,5,6,7,8,9,10]
#for L in range(1,11,1):
for L in Ls:
    loss = BinaryCrossentropy(from_logits=False)
    #out_act = keras.activations.linear
    model_kwargs = {"Phi_sizes":(100,100,128), "F_sizes":(100,100,100),
                    "F_acts":"relu","Phi_acts":"relu",
                    "F_dropouts":0.1,
                    "input_dim":2,
                    "output_dim":1,
                    "output_act":"sigmoid",
                    "patience":5,
                    "optimizer":Adam(1e-3),
                    "loss":loss,
                    "summary":True}
    model_kwargs['spectral'] = True
    model_kwargs['lip_const'] = float(L)
    model_kwargs['eps_bjorck'] = None
    model_kwargs['bound_only'] = True
    train_kwargs = {"epochs":200,"verbose":1,"batch_size":10000}
    efn, auc, efn_fp, efn_tp, threshs = train_efn((train,train_labels),
                                                  (test,test_labels),
                                                  (val,val_labels),
                                                  model_kwargs,
                                                  train_kwargs,
                                                  plot=True)
    efn.save(f"keras_models_wta_quarkGluon/spectralEFN_L{int(L)}_zpT_hadlevel.keras")
    del efn
    gc.collect()

In [ ]:
# EFN without spectral normalization
loss = BinaryCrossentropy(from_logits=False)
#out_act = keras.activations.linear
model_kwargs = {"Phi_sizes":(60,60,60), "F_sizes":(80,80,80),
                "F_dropouts":0.1,
                "input_dim":2,
                "output_dim":1,"output_act":"sigmoid",
                "patience":5,
                "optimizer":Adam(1e-3),
                "loss":loss,
                "summary":True}
model_kwargs['spectral'] = False
train_kwargs = {"epochs":200,"verbose":1,"batch_size":10000}
efn, auc, efn_fp, efn_tp, threshs = train_efn((train,train_labels),
                                              (test,test_labels),
                                              (val,val_labels),
                                              model_kwargs,
                                              train_kwargs,
                                              plot=True)
efn.save(f"keras_models_wta_quarkGluon/EFN_zpT_hadlevel.keras")
del efn
gc.collect()

In [ ]:
# PFN without spectral normalization
loss = BinaryCrossentropy(from_logits=False)
#out_act = keras.activations.linear
model_kwargs = {"Phi_sizes":(60,60,60), "F_sizes":(80,80,80),
                "F_dropouts":0.1,
                "input_dim":3,
                "output_dim":1,"output_act":"sigmoid",
                "patience":5,
                "optimizer":Adam(1e-3),
                "loss":loss,
                "summary":True}
model_kwargs['spectral'] = False
train_kwargs = {"epochs":200,"verbose":1,"batch_size":10000}
efn, auc, efn_fp, efn_tp, threshs = train_pfn((train,train_labels),
                                              (test,test_labels),
                                              (val,val_labels),
                                              model_kwargs,
                                              train_kwargs,
                                              plot=True)
efn.save(f"keras_models_wta_quarkGluon/PFN_zpT_hadlevel.keras")
del efn
gc.collect()

In [ ]:
# PFN with spectral normalization
L = 1
loss = BinaryCrossentropy(from_logits=False)
#out_act = keras.activations.linear
model_kwargs = {"Phi_sizes":(60,60,60), "F_sizes":(80,80,80),
                "F_dropouts":0.1,
                "input_dim":3,
                "output_dim":1,"output_act":"sigmoid",
                "patience":5,
                "optimizer":Adam(1e-3),
                "loss":loss,
                "summary":True}
model_kwargs['spectral'] = True
model_kwargs['lip_const'] = float(L)
model_kwargs['eps_bjorck'] = None
model_kwargs['bound_only'] = True
train_kwargs = {"epochs":200,"verbose":1,"batch_size":10000}
efn, auc, efn_fp, efn_tp, threshs = train_pfn((train,train_labels),
                                              (test,test_labels),
                                              (val,val_labels),
                                              model_kwargs,
                                              train_kwargs,
                                              plot=True)
efn.save(f"keras_models_wta_quarkGluon/spectralPFN_L{L}_zpT_hadlevel.keras")
del efn
gc.collect()

# Parton Level

In [ ]:
base="/uscms/home/sbrightt/nobackup/jets-ml/datasets/safeIncalculable_v2/"
nmax = 200000

hdata = get_data('q',hlevel=False,efrac=True,base=base,nmax=nmax,wta=True)
htrain, htest = train_test_split(hdata,train_size=0.5,random_state=rng1)
htest, hval = train_test_split(htest,train_size=0.5,random_state=rng2)

pdata = get_data('g',hlevel=False,efrac=True,base=base,nmax=nmax,wta=True)
ptrain, ptest = train_test_split(pdata,train_size=0.5,random_state=rng1)
ptest, pval = train_test_split(ptest,train_size=0.5,random_state=rng2)

train = np.concatenate((htrain,ptrain),axis=0)
train_labels = np.concatenate((np.ones((htrain.shape[0],1)),np.zeros((ptrain.shape[0],1))),axis=0)
perm = np.random.permutation(train.shape[0])
train = train[perm]
train_labels = train_labels[perm]

test = np.concatenate((htest,ptest),axis=0)
test_labels = np.concatenate((np.ones((htest.shape[0],1)),np.zeros((ptest.shape[0],1))),axis=0)
perm = np.random.permutation(test.shape[0])
test = test[perm]
test_labels = test_labels[perm]

val = np.concatenate((hval,pval),axis=0)
val_labels = np.concatenate((np.ones((hval.shape[0],1)),np.zeros((pval.shape[0],1))),axis=0)
perm = np.random.permutation(val.shape[0])
val = val[perm]
val_labels = val_labels[perm]

del hdata, pdata, htrain, htest, hval, ptrain, ptest, pval, perm

In [ ]:
# EFN with spectral normalization
for i in range(1):
    loss = BinaryCrossentropy(from_logits=False)
    #out_act = keras.activations.linear
    model_kwargs = {"Phi_sizes":(60,60,60), "F_sizes":(80,80,80),
                    "F_dropouts":0.1,
                    "input_dim":2,
                    "output_dim":1,"output_act":"sigmoid",
                    "patience":5,
                    "optimizer":Adam(1e-4),
                    "loss":loss,
                    "summary":True}
    model_kwargs['spectral'] = True
    train_kwargs = {"epochs":200,"verbose":1,"batch_size":10000}
    efn, auc, efn_fp, efn_tp, threshs = train_efn((train,train_labels),
                                                  (test,test_labels),
                                                  (val,val_labels),
                                                  model_kwargs,
                                                  train_kwargs,
                                                  plot=True)
    efn.save(f"keras_models_wta_quarkGluon/spectralEFN_zpT_partlevel_iter{i}.keras")
    del efn
    gc.collect()

In [ ]:
# spectral EFN L >= 1
#Ls = [20,30,40,50,60,70,80,90,
#      100,200,300,400,500,600,700,800,900,
#      1000,2000,3000,4000,5000,6000,7000,8000,9000,10000]
Ls = [2,3,4,5,6,7,8,9,10]
#for L in range(1,11,1):
for L in Ls:
    loss = BinaryCrossentropy(from_logits=False)
    #out_act = keras.activations.linear
    model_kwargs = {"Phi_sizes":(100,100,128), "F_sizes":(100,100,100),
                    "F_acts":"relu","Phi_acts":"relu",
                    "F_dropouts":0.1,
                    "input_dim":2,
                    "output_dim":1,
                    "output_act":"sigmoid",
                    "patience":5,
                    "optimizer":Adam(1e-3),
                    "loss":loss,
                    "summary":True}
    model_kwargs['spectral'] = True
    model_kwargs['lip_const'] = float(L)
    model_kwargs['eps_bjorck'] = None
    model_kwargs['bound_only'] = True
    train_kwargs = {"epochs":200,"verbose":1,"batch_size":10000}
    efn, auc, efn_fp, efn_tp, threshs = train_efn((train,train_labels),
                                                  (test,test_labels),
                                                  (val,val_labels),
                                                  model_kwargs,
                                                  train_kwargs,
                                                  plot=True)
    efn.save(f"keras_models_wta_quarkGluon/spectralEFN_L{int(L)}_zpT_partlevel.keras")
    del efn
    gc.collect()

In [ ]:
# EFN without spectral normalization
loss = BinaryCrossentropy(from_logits=False)
#out_act = keras.activations.linear
model_kwargs = {"Phi_sizes":(60,60,60), "F_sizes":(80,80,80),
                "F_dropouts":0.1,
                "input_dim":2,
                "output_dim":1,"output_act":"sigmoid",
                "patience":5,
                "optimizer":Adam(1e-3),
                "loss":loss,
                "summary":True}
model_kwargs['spectral'] = False
train_kwargs = {"epochs":200,"verbose":1,"batch_size":10000}
efn, auc, efn_fp, efn_tp, threshs = train_efn((train,train_labels),
                                              (test,test_labels),
                                              (val,val_labels),
                                              model_kwargs,
                                              train_kwargs,
                                              plot=True)
efn.save(f"keras_models_wta_quarkGluon/EFN_zpT_partlevel.keras")
del efn
gc.collect()

In [ ]:
# PFN without spectral normalization
loss = BinaryCrossentropy(from_logits=False)
#out_act = keras.activations.linear
model_kwargs = {"Phi_sizes":(60,60,60), "F_sizes":(80,80,80),
                "F_dropouts":0.1,
                "input_dim":3,
                "output_dim":1,"output_act":"sigmoid",
                "patience":5,
                "optimizer":Adam(1e-3),
                "loss":loss,
                "summary":True}
model_kwargs['spectral'] = False
train_kwargs = {"epochs":200,"verbose":1,"batch_size":10000}
efn, auc, efn_fp, efn_tp, threshs = train_pfn((train,train_labels),
                                              (test,test_labels),
                                              (val,val_labels),
                                              model_kwargs,
                                              train_kwargs,
                                              plot=True)
efn.save(f"keras_models_wta_quarkGluon/PFN_zpT_partlevel.keras")
del efn
gc.collect()

In [ ]:
# PFN with spectral normalization
L = 1
loss = BinaryCrossentropy(from_logits=False)
#out_act = keras.activations.linear
model_kwargs = {"Phi_sizes":(60,60,60), "F_sizes":(80,80,80),
                "F_dropouts":0.1,
                "input_dim":3,
                "output_dim":1,"output_act":"sigmoid",
                "patience":5,
                "optimizer":Adam(1e-3),
                "loss":loss,
                "summary":True}
model_kwargs['spectral'] = True
model_kwargs['lip_const'] = float(L)
model_kwargs['eps_bjorck'] = None
model_kwargs['bound_only'] = True
train_kwargs = {"epochs":200,"verbose":1,"batch_size":10000}
efn, auc, efn_fp, efn_tp, threshs = train_pfn((train,train_labels),
                                              (test,test_labels),
                                              (val,val_labels),
                                              model_kwargs,
                                              train_kwargs,
                                              plot=True)
efn.save(f"keras_models_wta_quarkGluon/spectralPFN_L{L}_zpT_partlevel.keras")
del efn
gc.collect()

# Evaluation

In [ ]:
import h5py
base = "/uscms/home/sbrightt/nobackup/jets-ml/datasets/safeIncalculable/"
with h5py.File(base+"h2qq_WTA_testSet_inputs_v2_pTinclusive_R1.0.h5","r") as f:
    zq = f['test'][:,:,:1]
    phat_q = f['test'][:,:,1:]
    label_q = f['labels'][()][:,0]

with h5py.File(base+"h2gg_WTA_testSet_inputs_v2_pTinclusive_R1.0.h5","r") as f:
    zg = f['test'][:,:,:1]
    phat_g = f['test'][:,:,1:]
    label_g = f['labels'][()][:,0]

zq_part = zq[label_q==0]
phat_q_part = phat_q[label_q==0]
zq_had = zq[label_q==1]
phat_q_had = phat_q[label_q==1]

zg_part = zg[label_g==0]
phat_g_part = phat_g[label_g==0]
zg_had = zg[label_g==1]
phat_g_had = phat_g[label_g==1]

z_part = np.concatenate((zq_part,zg_part),axis=0)
phat_part = np.concatenate((phat_q_part,phat_g_part),axis=0)
labels_part = np.concatenate((np.ones(zq_part.shape[0]),np.zeros(zg_part.shape[0])),axis=0)

z_had = np.concatenate((zq_had,zg_had),axis=0)
phat_had = np.concatenate((phat_q_had,phat_g_had),axis=0)
labels_had = np.concatenate((np.ones(zq_had.shape[0]),np.zeros(zg_had.shape[0])),axis=0)

def eval_model(modName,z,p,path="keras_models_wta_quarkGluon/",bs=1000):
    mod = keras.models.load_model(path+"/"+modName)
    if "PFN" in modName:
        preds = mod.predict(np.concatenate([z,p],axis=-1),batch_size=bs)[:,0]
    else:
        preds = mod.predict([z,p],batch_size=bs)[:,0]
    del mod
    return preds

In [ ]:
base = "keras_models_wta_quarkGluon/"
Ls = [2,3,4,5,6,7,8,9,10,20,30,40,50,60,70,80,90,100,200,300,400,500,600,700,800,900,1000,2000,3000,4000,5000,6000,7000,8000,9000,10000]
models_hadlevel = ["EFN_zpT_hadlevel.keras","PFN_zpT_hadlevel.keras","spectralEFN_L1_zpT_hadlevel.keras","spectralPFN_L1_zpT_hadlevel.keras"]+ \
                  [f"spectralEFN_L{L}_zpT_hadlevel.keras" for L in Ls]
models_partlevel = ["EFN_zpT_partlevel.keras","PFN_zpT_partlevel.keras","spectralEFN_L1_zpT_partlevel.keras","spectralPFN_L1_zpT_partlevel.keras"]+ \
                   [f"spectralEFN_L{L}_zpT_partlevel.keras" for L in Ls]
modKeys = ["EFN","PFN","spectralEFN","spectralPFN"] + [f"spectralEFN_L{L}" for L in Ls]

# eval had-level models on had-level data
print("evaluating hadron-level models on hadron-level data")
np.save(f"{base}/evaluations/labels_had.npy",labels_had)
for i,mod in enumerate(models_hadlevel):
    preds = eval_model(mod,z_had,phat_had,path=base)
    np.save(f"{base}/evaluations/preds_hadData_hadModel_{modKeys[i]}.npy",preds)
    del preds

# eval part-level models on part-level data
print("evaluating parton-level models on parton-level data")
np.save(f"{base}/evaluations/labels_part.npy",labels_part)
for i,mod in enumerate(models_partlevel):
    preds = eval_model(mod,z_part,phat_part,path=base)
    np.save(f"{base}/evaluations/preds_partData_partModel_{modKeys[i]}.npy",preds)
    del preds

# eval part-level models on had-level data
print("evaluating parton-level models on hadron-level data")
for i,mod in enumerate(models_partlevel):
    preds = eval_model(mod,z_had,phat_had,path=base)
    np.save(f"{base}/evaluations/preds_hadData_partModel_{modKeys[i]}.npy",preds)
    del preds